In [1]:
import time
import os

import argparse

from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import dask_cudf
import dask
from distributed import wait

import cupy as cp
from cuml.feature_extraction.text import HashingVectorizer
from cuml.dask.feature_extraction.text import TfidfTransformer

from nltk.corpus import stopwords

STOPWORDS = stopwords.words('english')
PUNCTUATIONS = [x for x in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']

FILES = ['/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Home_Entertainment_v1_00.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Office_Products_v1_00.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Kitchen_v1_00.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Mobile_Apps_v1_00.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Books_v1_02.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Musical_Instruments_v1_00.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Music_v1_00.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Digital_Video_Games_v1_00.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Grocery_v1_00.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Electronics_v1_00.tsv', '/scratch1/rgoli/aws_customer_reviews/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv']

In [3]:
data = dask_cudf.read_csv(FILES,sep='\t',columns=["review_body"],
                                 storage_options={'anon': True})

In [4]:
data.shape

(Delayed('int-1414420f-8aa3-431f-952e-8c6ac2b904fc'), 15)

In [6]:
data.size

dd.Scalar<size-ag..., dtype=int64>

In [8]:
def text_preprocessor(data, column_name, PUNCTUATIONS,STOPWORDS):
    data = data[data[column_name].notnull()]
    data[column_name] = data[column_name] \
                            .str.lower() \
                            .str.replace_tokens(PUNCTUATIONS, " ") \
                            .str.replace_tokens(STOPWORDS, "") \
                            .str.normalize_spaces() \
                            .str.strip()
    return data

def hashing_vectorizer(data, column_name):
    vectorizer = HashingVectorizer(stop_words=None, preprocessor=None)
    # Meta is an empty dataframe matches the dtypes and columns of the output
    meta = dask.array.from_array(cp.sparse.csr_matrix(cp.zeros(1, dtype=cp.float32)))
    hashing_vectorized = data[column_name].map_partitions(vectorizer.fit_transform, meta=meta).astype(cp.float32)
    hashing_vectorized = hashing_vectorized.persist()
    wait(hashing_vectorized);
    return hashing_vectorized


def tfidf_transformer(data, client):
    multi_gpu_transformer = TfidfTransformer(client=client)
    result = multi_gpu_transformer.fit_transform(data)
    result = result.persist()
    wait(result);
    result.compute_chunk_sizes()
    return result

In [10]:
data = text_preprocessor(data, "review_body", PUNCTUATIONS, STOPWORDS)

In [11]:
result = hashing_vectorizer(data, "review_body")

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/rgoli/.conda/envs/rapids-22.02/include/rmm/mr/device/cuda_memory_resource.hpp

In [ ]:
result = tfidf_transformer(result, client)